<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Introduction" data-toc-modified-id="Introduction-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Introduction</a></span><ul class="toc-item"><li><span><a href="#Don't-use-mutable-objects-as-default-arguments-for-functions!" data-toc-modified-id="Don't-use-mutable-objects-as-default-arguments-for-functions!-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Don't use mutable objects as default arguments for functions!</a></span></li><li><span><a href="#When-mutable-contents-of-immutable-tuples-aren't-so-mutable" data-toc-modified-id="When-mutable-contents-of-immutable-tuples-aren't-so-mutable-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>When mutable contents of immutable tuples aren't so mutable</a></span><ul class="toc-item"><li><span><a href="#But-what-if-we-put-a-mutable-object-into-the-immutable-tuple?-Well,-modification-works,-but-we-also-get-a-TypeError-at-the-same-time." data-toc-modified-id="But-what-if-we-put-a-mutable-object-into-the-immutable-tuple?-Well,-modification-works,-but-we-also-get-a-TypeError-at-the-same-time.-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>But what if we put a mutable object into the immutable tuple? Well, modification works, but we <strong>also</strong> get a <code>TypeError</code> at the same time.</a></span></li><li><span><a href="#Explanation" data-toc-modified-id="Explanation-1.2.2"><span class="toc-item-num">1.2.2&nbsp;&nbsp;</span>Explanation</a></span></li><li><span><a href="#One-more-note-about-the-immutable-status-of-tuples.-Tuples-are-famous-for-being-immutable.-However,-how-comes-that-this-code-works?" data-toc-modified-id="One-more-note-about-the-immutable-status-of-tuples.-Tuples-are-famous-for-being-immutable.-However,-how-comes-that-this-code-works?-1.2.3"><span class="toc-item-num">1.2.3&nbsp;&nbsp;</span>One more note about the <code>immutable</code> status of tuples. Tuples are famous for being immutable. However, how comes that this code works?</a></span></li></ul></li><li><span><a href="#Interning-of-compile-time-constants-vs.-run-time-expressions" data-toc-modified-id="Interning-of-compile-time-constants-vs.-run-time-expressions-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Interning of compile-time constants vs. run-time expressions</a></span></li></ul></li><li><span><a href="#References" data-toc-modified-id="References-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>References</a></span></li></ul></div>

# Introduction
<hr style = "border:2px solid black" ></hr>

<div class="alert alert-warning">
<font color=black>

**What?** A collection of not-so-obvious Python stuff

</font>
</div>

## Don't use mutable objects as default arguments for functions!

Don't use mutable objects (e.g., dictionaries, lists, sets, etc.) as default arguments for functions! You might expect that a new list is created every time when we call the function without providing an argument for the default parameter, but this is not the case: **Python will create the mutable object (default parameter) the first time the function is defined - not when it is called**, see the following code:

(Original source: [http://docs.python-guide.org/en/latest/writing/gotchas/](http://docs.python-guide.org/en/latest/writing/gotchas/)

In [15]:
def append_to_list(value, def_list=[]):
    def_list.append(value)
    return def_list

my_list = append_to_list(1)
print(my_list)

my_other_list = append_to_list(2)
print(my_other_list)

[1]
[1, 2]


Another good example showing that demonstrates that default arguments are created when the function is created (**and not when it is called!**):

In [16]:
import time
def report_arg(my_default=time.time()):
    print(my_default)

report_arg()

time.sleep(5)

report_arg()

1528560045.3962939
1528560045.3962939


## When mutable contents of immutable tuples aren't so mutable

As we all know, tuples are immutable objects in Python, right!? But what happens if they contain mutable objects? 

First, let us have a look at the expected behavior: a `TypeError` is raised if we try to modify immutable types in a tuple: 

In [27]:
tup = (1,)
tup[0] += 1

TypeError: 'tuple' object does not support item assignment

### But what if we put a mutable object into the immutable tuple? Well, modification works, but we **also** get a `TypeError` at the same time.

In [28]:
tup = ([],)
print('tup before: ', tup)
tup[0] += [1]

tup before:  ([],)


TypeError: 'tuple' object does not support item assignment

In [ ]:
print('tup after: ', tup)

<br>
<br>
However, **there are ways** to modify the mutable contents of the tuple without raising the `TypeError`, the solution is the `.extend()` method, or alternatively `.append()` (for lists):

In [29]:
tup = ([],)
print('tup before: ', tup)
tup[0].extend([1])
print('tup after: ', tup)

tup before:  ([],)
tup after:  ([1],)


In [30]:
tup = ([],)
print('tup before: ', tup)
tup[0].append(1)
print('tup after: ', tup)

tup before:  ([],)
tup after:  ([1],)


### Explanation

**A. Jesse Jiryu Davis** has a nice explanation for this phenomenon (Original source: [http://emptysqua.re/blog/python-increment-is-weird-part-ii/](http://emptysqua.re/blog/python-increment-is-weird-part-ii/))

If we try to extend the list via `+=` *"then the statement executes `STORE_SUBSCR`, which calls the C function `PyObject_SetItem`, which checks if the object supports item assignment. In our case the object is a tuple, so `PyObject_SetItem` throws the `TypeError`. Mystery solved."*

### One more note about the `immutable` status of tuples. Tuples are famous for being immutable. However, how comes that this code works?

In [31]:
my_tup = (1,)
my_tup += (4,)
my_tup = my_tup + (5,)
print(my_tup)

(1, 4, 5)


What happens "behind" the curtains is that the tuple is not modified, but a new object is generated every time, which will inherit the old "name tag":

In [32]:
my_tup = (1,)
print(id(my_tup))
my_tup += (4,)
print(id(my_tup))
my_tup = my_tup + (5,)
print(id(my_tup))

4486707912
4485211784
4486955152


## Interning of compile-time constants vs. run-time expressions

This might not be particularly useful, but it is nonetheless interesting: Python's interpreter is interning compile-time constants but not run-time expressions (note that this is implementation-specific).

(Original source: [Stackoverflow](http://stackoverflow.com/questions/15541404/python-string-interning))

Let us have a look at the simple example below. Here we are creating 3 variables and assign the value "Hello" to them in different ways before we test them for identity.

In [42]:
hello1 = 'Hello'

hello2 = 'Hell' + 'o'

hello3 = 'Hell'
hello3 = hello3 + 'o'

print('hello1 is hello2:', hello1 is hello2)
print('hello1 is hello3:', hello1 is hello3)

hello1 is hello2: True
hello1 is hello3: False


Now, how does it come that the first expression evaluates to true, but the second does not?  To answer this question, we need to take a closer look at the underlying byte codes:

In [43]:
import dis
def hello1_func():
    s = 'Hello'
    return s
dis.dis(hello1_func)

  3           0 LOAD_CONST               1 ('Hello')
              2 STORE_FAST               0 (s)

  4           4 LOAD_FAST                0 (s)
              6 RETURN_VALUE


In [44]:
def hello2_func():
    s = 'Hell' + 'o'
    return s
dis.dis(hello2_func)

  2           0 LOAD_CONST               3 ('Hello')
              2 STORE_FAST               0 (s)

  3           4 LOAD_FAST                0 (s)
              6 RETURN_VALUE


In [45]:
def hello3_func():
    s = 'Hell'
    s = s + 'o'
    return s
dis.dis(hello3_func)

  2           0 LOAD_CONST               1 ('Hell')
              2 STORE_FAST               0 (s)

  3           4 LOAD_FAST                0 (s)
              6 LOAD_CONST               2 ('o')
              8 BINARY_ADD
             10 STORE_FAST               0 (s)

  4          12 LOAD_FAST                0 (s)
             14 RETURN_VALUE


<br>
It looks like that `'Hello'` and `'Hell'` + `'o'` are both evaluated and stored as `'Hello'` at compile-time, whereas the third version   
`s = 'Hell'`  
`s = s + 'o'` seems to not be interned.  Let us quickly confirm the behavior with the following code:

In [46]:
print(hello1_func() is hello2_func())
print(hello1_func() is hello3_func())

True
False


Finally, to show that this hypothesis is the answer to this rather unexpected observation, let us `intern` the value manually:

In [47]:
import sys

print(hello1_func() is sys.intern(hello3_func()))

True


# References
<hr style = "border:2px solid black" ></hr>

<div class="alert alert-warning">
<font color=black>

- https://nbviewer.org/github/rasbt/python_reference/blob/master/tutorials/not_so_obvious_python_stuff.ipynb?create=1

</font>
</div>